<a href="https://colab.research.google.com/github/neftali-lopez-elizondo/Decimal-Microtonal-Scale/blob/main/Convertidor_MIDI_a_MIDI_Decimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
from mido import MidiFile, MidiTrack, Message

# --------------------------
# Configuración del sistema musical decimal
# --------------------------
# Razón de la octava para el sistema de 10 tonos
OCTAVE_RATIO = 2**(1/10)
# Frecuencia de referencia (La 440 Hz)
A4_FREQ = 440.0
# Nota de referencia en el sistema de 10 tonos
LA_TONE_DECIMAL = 8
# Nota de referencia en el sistema MIDI estándar
A4_MIDI_NOTE = 69
# Razón de la octava para el sistema de 12 tonos
OCTAVE_RATIO_12 = 2**(1/12)

def get_standard_freq(midi_note):
    """Calcula la frecuencia en Hz de una nota MIDI estándar."""
    return A4_FREQ * (OCTAVE_RATIO_12 ** (midi_note - A4_MIDI_NOTE))

def get_closest_decimal_note(standard_freq):
    """
    Convierte una frecuencia estándar a la nota más cercana
    en el sistema de 10 tonos.
    """
    # Fórmula de la nota decimal
    # tone = 8 + 10 * log2(frecuencia / A4_FREQ)
    decimal_tone = LA_TONE_DECIMAL + 10 * math.log2(standard_freq / A4_FREQ)
    return decimal_tone

def get_midi_note_from_decimal(decimal_tone):
    """
    Convierte un tono decimal a la nota MIDI estándar más cercana.
    Esto es para crear un nuevo archivo MIDI que se pueda reproducir.
    """
    # Fórmula para la nota MIDI
    # midi_note = 69 + 12 * log2(frecuencia_decimal / A4_FREQ)
    decimal_freq = A4_FREQ * (OCTAVE_RATIO ** (decimal_tone - LA_TONE_DECIMAL))
    midi_note = A4_MIDI_NOTE + 12 * math.log2(decimal_freq / A4_FREQ)

    # Redondeamos a la nota entera más cercana, ya que MIDI solo usa enteros.
    return int(round(midi_note))

# --------------------------
# Clase principal del convertidor
# --------------------------
class MidiDecimalConverter:
    """
    Un convertidor que toma un archivo MIDI estándar y crea un nuevo
    archivo MIDI en el sistema musical decimal.
    """
    def __init__(self, input_path, output_path):
        self.input_path = input_path
        self.output_path = output_path
        self.mid = MidiFile(self.input_path)

    def convert_to_decimal_midi(self):
        """
        Lee el archivo de entrada y crea un nuevo archivo MIDI
        con las notas convertidas al sistema decimal.
        """
        print(f"Iniciando conversión de '{self.input_path}'...")

        # Crea un nuevo archivo MIDI para el resultado
        new_midi = MidiFile(ticks_per_beat=self.mid.ticks_per_beat)

        for track in self.mid.tracks:
            new_track = MidiTrack()
            new_midi.tracks.append(new_track)

            for msg in track:
                if msg.is_meta:
                    # Copia mensajes meta como el tempo, el tiempo, etc.
                    new_track.append(msg)
                elif msg.type in ['note_on', 'note_off']:
                    # 1. Convierte la nota MIDI estándar a su frecuencia
                    standard_freq = get_standard_freq(msg.note)

                    # 2. Convierte la frecuencia a un tono decimal
                    decimal_tone = get_closest_decimal_note(standard_freq)

                    # 3. Vuelve a convertir el tono decimal a la nota MIDI más cercana
                    new_midi_note = get_midi_note_from_decimal(decimal_tone)

                    # 4. Crea un nuevo mensaje MIDI con la nota convertida
                    new_msg = Message(
                        msg.type,
                        note=new_midi_note,
                        velocity=msg.velocity,
                        time=msg.time,
                        channel=msg.channel
                    )
                    new_track.append(new_msg)
                else:
                    # Copia otros mensajes del canal
                    new_track.append(msg)

        # Guarda el nuevo archivo MIDI
        new_midi.save(self.output_path)
        print(f"✅ Conversión completa. Archivo guardado como '{self.output_path}'")

# --------------------------
# Ejemplo de uso
# --------------------------
if __name__ == "__main__":
    # Necesitas un archivo MIDI en la misma carpeta del script
    input_file = "mi_pieza_clasica.mid"
    output_file = "pieza_decimal.mid"

    try:
        converter = MidiDecimalConverter(input_file, output_file)
        converter.convert_to_decimal_midi()
    except FileNotFoundError:
        print(f"❌ Error: Asegúrate de tener un archivo MIDI llamado '{input_file}' en la misma carpeta.")

ModuleNotFoundError: No module named 'mido'